# **Importing Necessary Libraries**

In [ ]:
# Pandas
import pandas as pd
from IPython.display import display

# NumPy
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# Sci-Kit Learn
import sklearn
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error as mae
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer

# XGBoost
from xgboost import XGBClassifier

# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, Sequential
from keras.layers import Dropout, BatchNormalization, Activation, Dense

# Math Libraries
import random
from random import uniform
import math
import scipy
from scipy.stats import norm, t, chi2

# Pickle
import pickle

# LIME
!pip install lime
import lime
from lime import lime_tabular

# Print Confirmation
print("Setup Complete")

# **Importing Data, Imputation, and Interpolation**

In [ ]:
url = "https://raw.githubusercontent.com/csun365/Cardiovascular-Disease/main/cardiovascular_raw.csv"
df = pd.read_csv(url)

In [ ]:
df.isna().sum().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df["Cholesterol"] = df["Cholesterol"].replace([0], np.nan)

In [ ]:
df["RestingBP"] = df["RestingBP"].replace([0], np.median(df["RestingBP"]))

In [ ]:
df["Sex"] = df["Sex"] == "M"
df["ExerciseAngina"] = df["ExerciseAngina"] == "Y"
df = pd.get_dummies(df, columns=["ChestPainType", "RestingECG", "ST_Slope"])

In [ ]:
cols = df.columns.tolist()
idx = cols.index("HeartDisease")
cols = cols[:idx] + cols[idx+1:] + ["HeartDisease"]
df = df[cols]

In [ ]:
X_impute = []
Y_impute = []
for i in range(df.shape[0]):
  if df["Cholesterol"].isna()[i] == 0:
    X_impute.append(df.iloc[i,:-1])
    Y_impute.append(df.iloc[i,-1])
X_impute = np.array(X_impute)
Y_impute = np.array(Y_impute)

In [ ]:
X_t_impute, X_v_impute, y_t_impute, y_v_impute = train_test_split(X_impute, Y_impute, test_size=0.25, random_state=1)

In [ ]:
choices = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,20,30,40,50,100,200]
best_k = 1
best_accuracy = 0
for i in choices:
  KNC = KNeighborsClassifier(n_neighbors = i).fit(X_t_impute, y_t_impute)
  y_pred_impute = KNC.predict(X_v_impute)
  accuracy = accuracy_score(y_v_impute, y_pred_impute)
  if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_k = i
  print("K = " + str(i) + " : " + str(accuracy))
print("Best K: " + str(best_k))

In [ ]:
imputer = KNNImputer(n_neighbors=best_k)
imputed = imputer.fit_transform(df)
df["Cholesterol"] = imputed[:,3]

In [ ]:
df.to_csv("cardiovascular_imputed_interpolated.csv", index=False)

In [ ]:
url = "https://raw.githubusercontent.com/csun365/Cardiovascular-Disease/main/cardiovascular_imputed_interpolated.csv"
df = pd.read_csv(url)

In [ ]:
X = df.iloc[:,:-1].values.astype(float)
Y = df.iloc[:,-1].values

In [ ]:
X_t, X_v, y_t, y_v = train_test_split(X, Y, test_size=0.25, random_state=2)

print("Number of patients in train set: " + str(y_t.shape[0]))
print("Number of patients in validation set: " + str(y_v.shape[0]))

# **Exploratory Data Analysis**

In [ ]:
df_neg = df[df["HeartDisease"] == 0]
df_pos = df[df["HeartDisease"] == 1]

In [ ]:
X_t_mean = np.mean(X_t, axis=0)
X_t_std = np.std(X_t, axis=0)

X_t = (X_t - X_t_mean) / X_t_std
X_v = (X_v - X_t_mean) / X_t_std

In [ ]:
cols = [0,2,3,5,7]
for i in cols:
  print("Minimum " + str(df.columns[i]) + ": " + str(df.iloc[:,i].min()))
  print("Maximum " + str(df.columns[i]) + ": " + str(df.iloc[:,i].max()))
  print()

In [ ]:
def degrees_of_freedom(sigma1, n1, sigma2, n2):
  numerator = (sigma1 ** 2 / n1 + sigma2 ** 2 / n2) ** 2
  denominator = (sigma1 ** 2 / n1) ** 2 / (n1 - 1) + (sigma2 ** 2 / n2) ** 2 / (n2 - 1)
  return numerator / denominator

In [ ]:
# Two Sample 
pos_mean_age = np.mean(df_pos["Age"])
neg_mean_age = np.mean(df_neg["Age"])
pos_std_age = np.std(df_pos["Age"])
neg_std_age = np.std(df_neg["Age"])

t_age = (pos_mean_age - neg_mean_age) / (pos_std_age ** 2 / df_pos.shape[0] + neg_std_age ** 2 / df_neg.shape[0]) ** 0.5
print(t_age)
df_age = degrees_of_freedom(pos_std_age, df_pos.shape[0], neg_std_age, df_neg.shape[0])
print(df_age)
p_age = t.cdf(-t_age, df=df_age)
print(p_age)

In [ ]:
pos_mean_BP = np.mean(df_pos["RestingBP"])
neg_mean_BP = np.mean(df_neg["RestingBP"])
pos_std_BP = np.std(df_pos["RestingBP"])
neg_std_BP = np.std(df_neg["RestingBP"])

t_BP = (pos_mean_BP - neg_mean_BP) / (pos_std_BP ** 2 / df_pos.shape[0] + neg_std_BP ** 2 / df_neg.shape[0]) ** 0.5
print(t_BP)
df_BP = degrees_of_freedom(pos_std_BP, df_pos.shape[0], neg_std_BP, df_neg.shape[0])
print(df_BP)
p_BP = t.cdf(-t_BP, df=df_BP)
print(p_BP)

In [ ]:
pos_mean_chol = np.mean(df_pos["Cholesterol"])
neg_mean_chol = np.mean(df_neg["Cholesterol"])
pos_std_chol = np.std(df_pos["Cholesterol"])
neg_std_chol = np.std(df_neg["Cholesterol"])

t_chol = (pos_mean_chol - neg_mean_chol) / (pos_std_chol ** 2 / df_pos.shape[0] + neg_std_chol ** 2 / df_neg.shape[0]) ** 0.5
print(t_chol)
df_chol = degrees_of_freedom(pos_std_chol, df_pos.shape[0], neg_std_chol, df_neg.shape[0])
print(df_chol)
p_chol = t.cdf(-t_chol, df=df_chol)
print(p_chol)

In [ ]:
pos_mean_HR = np.mean(df_pos["MaxHR"])
neg_mean_HR = np.mean(df_neg["MaxHR"])
pos_std_HR = np.std(df_pos["MaxHR"])
neg_std_HR = np.std(df_neg["MaxHR"])

t_HR = (pos_mean_HR - neg_mean_HR) / (pos_std_HR ** 2 / df_pos.shape[0] + neg_std_HR ** 2 / df_neg.shape[0]) ** 0.5
print(t_HR)
df_HR = degrees_of_freedom(pos_std_HR, df_pos.shape[0], neg_std_HR, df_neg.shape[0])
print(df_HR)
p_HR = t.cdf(t_HR, df=df_HR)
print(p_HR)

In [ ]:
pos_mean_oldpeak = np.mean(df_pos["Oldpeak"])
neg_mean_oldpeak = np.mean(df_neg["Oldpeak"])
pos_std_oldpeak = np.std(df_pos["Oldpeak"])
neg_std_oldpeak = np.std(df_neg["Oldpeak"])

t_oldpeak = (pos_mean_oldpeak - neg_mean_oldpeak) / (pos_std_oldpeak ** 2 / df_pos.shape[0] + neg_std_oldpeak ** 2 / df_neg.shape[0]) ** 0.5
print(t_oldpeak)
df_oldpeak = degrees_of_freedom(pos_std_oldpeak, df_pos.shape[0], neg_std_oldpeak, df_neg.shape[0])
print(df_oldpeak)
p_oldpeak = t.cdf(-t_oldpeak, df=df_oldpeak)
print(p_oldpeak)

In [ ]:
cols = [0,2,3,5,7]
fig, ax = plt.subplots(2, 3, figsize=(15,6))
coords = [(i,j) for i in range(2) for j in range(3)]
counter = 0
for i in cols:
  ax[coords[counter]].set_title(df.columns[i])
  ax[coords[counter]].hist(df_pos[df.columns[i]], label="Positive", alpha = 0.7, color="r", bins=10)
  ax[coords[counter]].hist(df_neg[df.columns[i]], label="Negative", alpha = 0.7, color="b", bins=10)
  ax[coords[counter]].legend()
  counter += 1
ax[1,2].remove()
plt.savefig("EDA_means.png")
plt.show()

In [ ]:
# Chi-Square 
obs_chest_pain = np.array([[df_pos[df_pos["ChestPainType_ATA"] == 1].shape[0], df_neg[df_neg["ChestPainType_ATA"] == 1].shape[0]],
                           [df_pos[df_pos["ChestPainType_NAP"] == 1].shape[0], df_neg[df_neg["ChestPainType_NAP"] == 1].shape[0]],
                           [df_pos[df_pos["ChestPainType_ASY"] == 1].shape[0], df_neg[df_neg["ChestPainType_ASY"] == 1].shape[0]],
                           [df_pos[df_pos["ChestPainType_TA"] == 1].shape[0], df_neg[df_neg["ChestPainType_TA"] == 1].shape[0]]])

exp_chest_pain = np.zeros((obs_chest_pain.shape[0], obs_chest_pain.shape[1]))
for i in range(exp_chest_pain.shape[0]):
  for j in range(exp_chest_pain.shape[1]):
    exp_chest_pain[i][j] = np.sum(obs_chest_pain, axis=0)[j] * np.sum(obs_chest_pain, axis=1)[i] / np.sum(obs_chest_pain)

X2_chest_pain = np.sum(np.divide(np.square(obs_chest_pain - exp_chest_pain), exp_chest_pain))
print(X2_chest_pain)
p_chest_pain = chi2.cdf(-X2_chest_pain, df=3)
print(p_chest_pain)

In [ ]:
obs_ECG = np.array([[df_pos[df_pos["RestingECG_Normal"] == 1].shape[0], df_neg[df_neg["RestingECG_Normal"] == 1].shape[0]],
                           [df_pos[df_pos["RestingECG_ST"] == 1].shape[0], df_neg[df_neg["RestingECG_ST"] == 1].shape[0]],
                           [df_pos[df_pos["RestingECG_LVH"] == 1].shape[0], df_neg[df_neg["RestingECG_LVH"] == 1].shape[0]]])

exp_ECG = np.zeros((obs_ECG.shape[0], obs_ECG.shape[1]))
for i in range(exp_ECG.shape[0]):
  for j in range(exp_ECG.shape[1]):
    exp_ECG[i][j] = np.sum(obs_ECG, axis=0)[j] * np.sum(obs_ECG, axis=1)[i] / np.sum(obs_ECG)

X2_ECG = np.sum(np.divide(np.square(obs_ECG - exp_ECG), exp_ECG))
print(X2_ECG)
p_ECG = chi2.cdf(-X2_ECG, df=2)
print(p_ECG)

In [ ]:
obs_ST = np.array([[df_pos[df_pos["ST_Slope_Up"] == 1].shape[0], df_neg[df_neg["ST_Slope_Up"] == 1].shape[0]],
                           [df_pos[df_pos["ST_Slope_Flat"] == 1].shape[0], df_neg[df_neg["ST_Slope_Flat"] == 1].shape[0]],
                           [df_pos[df_pos["ST_Slope_Down"] == 1].shape[0], df_neg[df_neg["ST_Slope_Down"] == 1].shape[0]]])

exp_ST = np.zeros(obs_ST.shape)
for i in range(exp_ST.shape[0]):
  for j in range(exp_ST.shape[1]):
    exp_ST[i][j] = np.sum(obs_ST, axis=0)[j] * np.sum(obs_ST, axis=1)[i] / np.sum(obs_ST)

X2_ST = np.sum(np.divide(np.square(obs_ST - exp_ST), exp_ST))
print(X2_ST)
p_ST = chi2.cdf(-X2_ST, df=2)
print(p_ST)

In [ ]:
# Two Propotion
pos_M = df[(df["HeartDisease"] == 1) & (df["Sex"] == 1)].shape[0]
pos_F = df[(df["HeartDisease"] == 1) & (df["Sex"] == 0)].shape[0]
tot_M = df[df["Sex"] == 1].shape[0]
tot_F = df[df["Sex"] == 0].shape[0]
prop_M = pos_M / tot_M
prop_F = pos_F / tot_F
prop_pool_sex = (pos_M + pos_F) / (tot_M + tot_F)

z_sex = (prop_M - prop_F) / ((prop_pool_sex * (1 - prop_pool_sex) * (1 / tot_M + 1 / tot_F)) ** 0.5)
print(z_sex)
p_sex = norm.cdf(-z_sex)
print(p_sex)

In [ ]:
pos_BShigh = df[(df["HeartDisease"] == 1) & (df["FastingBS"] == 1)].shape[0]
pos_BSlow = df[(df["HeartDisease"] == 1) & (df["FastingBS"] == 0)].shape[0]
tot_BShigh = df[df["FastingBS"] == 1].shape[0]
tot_BSlow = df[df["FastingBS"] == 0].shape[0]
prop_BShigh = pos_BShigh / tot_BShigh
prop_BSlow = pos_BSlow / tot_BSlow
prop_pool_BS = (pos_M + pos_F) / (tot_M + tot_F)

z_BS = (prop_BShigh - prop_BSlow) / ((prop_pool_BS * (1 - prop_pool_BS) * (1 / tot_BShigh + 1 / tot_BSlow)) ** 0.5)
print(z_BS)
p_BS = norm.cdf(-z_BS)
print(p_BS)

In [ ]:
pos_angina = df[(df["HeartDisease"] == 1) & (df["ExerciseAngina"] == 1)].shape[0]
pos_none = df[(df["HeartDisease"] == 1) & (df["ExerciseAngina"] == 0)].shape[0]
tot_angina = df[df["ExerciseAngina"] == 1].shape[0]
tot_none = df[df["ExerciseAngina"] == 0].shape[0]
prop_angina= pos_angina / tot_angina
prop_none = pos_none / tot_none
prop_pool_angina = (pos_angina + pos_none) / (tot_angina + tot_none)

z_angina = (prop_angina - prop_none) / ((prop_pool_angina * (1 - prop_pool_angina) * (1 / tot_angina + 1 / tot_none)) ** 0.5)
print(z_angina)
p_angina = norm.cdf(-z_angina)
print(p_angina)

# **Predictive Modeling**

In [ ]:
def f1_pd(y_pred, y_true):
  tp = np.sum(np.logical_and((y_pred==1), (y_true==1)))
  fp = np.sum(np.logical_and((y_pred==1), (y_true==0)))
  tn = np.sum(np.logical_and((y_pred==0), (y_true==0)))
  fn = np.sum(np.logical_and((y_pred==0), (y_true==1)))
  
  r = tp/(tp + fn) 
  p = tp/(tp + fp) 
  
  f1 = 2 * p * r / (p + r) 
  return f1

In [ ]:
# Logistic Regression Model (LR)
model_lr = LogisticRegression(random_state=0)
model_lr.fit(X_t, y_t)
probs_lr = model_lr.predict_proba(X_v)
predictions_lr = probs_lr[:,1] >= probs_lr[:,0]
binary_accuracy_lr = accuracy_score(y_v, predictions_lr)
f1_lr = f1_pd(predictions_lr, y_v)
print("Logistic Regression Binary Accuracy: " + str(binary_accuracy_lr))
print("Logistic Regression F1 Score: " + str(f1_lr))

fpr, tpr, _ = sklearn.metrics.roc_curve(y_v, probs_lr[:,1])
plt.plot(fpr,tpr,label="AUC = " + str(sklearn.metrics.roc_auc_score(y_v, probs_lr[:,1])))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [ ]:
# Random Forest Model (RF)
model_rf = RandomForestClassifier(n_estimators=100,random_state=0)
model_rf.fit(X_t, y_t)
probs_rf = model_rf.predict_proba(X_v)
predictions_rf = probs_rf[:,1] >= probs_rf[:,0]
binary_accuracy_rf = accuracy_score(y_v, predictions_rf)
f1_rf = f1_pd(predictions_rf,y_v)
print("RandomForestClassifier Binary Accuracy: " + str(binary_accuracy_rf))
print("RandomForestClassifier F1 Score: " + str(f1_rf))

fpr, tpr, _ = sklearn.metrics.roc_curve(y_v, probs_rf[:,1])
plt.plot(fpr,tpr,label="AUC = " + str(sklearn.metrics.roc_auc_score(y_v, probs_rf[:,1])))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [ ]:
# Extreme Gradient Boosting Model (XGB)
model_xgb = XGBClassifier(n_estimators=100,learning_rate=1e-1,n_jobs=4,random_state=0)
model_xgb.fit(X_t,y_t,early_stopping_rounds=100,eval_set=[(X_v,y_v)],verbose=False)
probs_xgb = model_xgb.predict_proba(X_v)
predictions_xgb = probs_xgb[:,1] >= probs_xgb[:,0]
binary_accuracy_xgb = accuracy_score(y_v, predictions_xgb)
f1_xgb = f1_pd(predictions_xgb, y_v)
print("XGBClassifier Binary Accuracy: " + str(binary_accuracy_xgb))
print("XGBClassifier F1 Score: " + str(f1_xgb))

fpr, tpr, _ = sklearn.metrics.roc_curve(y_v, probs_xgb[:,1])
plt.plot(fpr,tpr,label="AUC = " + str(sklearn.metrics.roc_auc_score(y_v, probs_xgb[:,1])))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [ ]:
# Support Vector Machine (SVM)
model_svm = SVC(C=1, kernel="rbf", random_state=0, probability=True)
model_svm.fit(X_t,y_t)
probs_svm = model_svm.predict_proba(X_v)
predictions_svm = probs_svm[:,1] >= probs_svm[:,0]
binary_accuracy_svm = accuracy_score(y_v, predictions_svm)
f1_svm = f1_pd(predictions_svm,y_v)
print("Support Vector Machine Binary Accuracy: " + str(binary_accuracy_svm))
print("Support Vector Machine F1 Score: " + str(f1_svm))

fpr, tpr, _ = sklearn.metrics.roc_curve(y_v, probs_svm[:,1])
plt.plot(fpr,tpr,label="AUC = " + str(sklearn.metrics.roc_auc_score(y_v, probs_svm[:,1])))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [ ]:
hidden_units = 64
dropout_rate = 0.4

layers_dimensions = [hidden_units,hidden_units,hidden_units,hidden_units]
initializer = tf.keras.initializers.GlorotNormal(seed=2) 
model_nn = keras.Sequential([
                            layers.BatchNormalization(input_shape=[X_t.shape[1]]),
                            layers.Dense(layers_dimensions[0],kernel_initializer=initializer,bias_initializer='zeros'), 
                            layers.BatchNormalization(),
                            layers.Activation("relu"),
                            layers.Dropout(rate=dropout_rate),
                            layers.Dense(layers_dimensions[1],kernel_initializer=initializer,bias_initializer='zeros'),
                            layers.BatchNormalization(),
                            layers.Activation("relu"),
                            layers.Dropout(rate=dropout_rate),
                            layers.Dense(layers_dimensions[2],kernel_initializer=initializer,bias_initializer='zeros'),
                            layers.BatchNormalization(),
                            layers.Activation("relu"),
                            layers.Dropout(rate=dropout_rate),
                            layers.Dense(layers_dimensions[3],kernel_initializer=initializer,bias_initializer='zeros'),
                            layers.BatchNormalization(),
                            layers.Activation("relu"),
                            layers.Dense(1,kernel_initializer=initializer,bias_initializer='zeros'),
                            layers.Activation("sigmoid")
])

optimizer = keras.optimizers.Adam(learning_rate=5e-3)
model_nn.compile(optimizer="adam",loss="binary_crossentropy",metrics=["binary_accuracy"])

In [ ]:
model_nn.summary()

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor="val_binary_accuracy", mode="max", min_delta=0.001, patience=50, restore_best_weights=True)
history_nn = model_nn.fit(X_t,y_t,validation_data=(X_v,y_v),callbacks=[early_stopping],batch_size=64,epochs=250,use_multiprocessing=True,verbose=1)
history_nn_pd = pd.DataFrame(history_nn.history)

In [ ]:
plt.plot(history_nn_pd["loss"])
plt.plot(history_nn_pd["val_loss"])
plt.show()

plt.plot(history_nn_pd["binary_accuracy"])
plt.plot(history_nn_pd["val_binary_accuracy"])
plt.show()

In [ ]:
model_nn.load_weights("/content/cardiovascular_model_weights_64_0.4_fourlayers.h5")

In [ ]:
y_pred = model_nn.predict(X_v)
y_pred = y_pred.reshape(-1)

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_v, y_pred)
plt.plot(fpr,tpr,label="AUC = " + str(sklearn.metrics.roc_auc_score(y_v, y_pred)))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [ ]:
y_pred = y_pred >= 0.5
print(np.sum(y_pred==y_v)/y_pred.shape[0])
f1_nn = f1_pd(y_pred, y_v)
print(f1_nn)

In [ ]:
def confusion(y_pred, y_true):
  tp = np.sum(np.logical_and((y_pred==1), (y_true==1)))
  fp = np.sum(np.logical_and((y_pred==1), (y_true==0)))
  tn = np.sum(np.logical_and((y_pred==0), (y_true==0)))
  fn = np.sum(np.logical_and((y_pred==0), (y_true==1)))
  return tp, fp, tn, fn

In [ ]:
confusion(y_pred, y_v)

In [ ]:
history_nn_pd.to_csv("cardio_model_history.csv", index=False)
model_nn.save_weights("cardiovascular_model_weights_64_0.4_fourlayers.h5")

In [ ]:
df.columns

In [ ]:
patient = np.array([40,0,110,100,1,175,0,2,0,0,0,1,0,1,0,0,0,1])
patient = (patient - X_t_mean) / X_t_std
diagnosis = model_nn.predict(patient.reshape(1,-1))
print(diagnosis)

# **K-Means Clustering and Local Interpretable Model-Agnostic Explanations (LIME)**

In [ ]:
# L2 Normalization for Clustering 
X_div = np.sum(X**2, axis=0)**0.5
X_norm = X/X_div

X_pos_clustering = X_norm[Y==1] # The X values of all of the "Unhealthy patients"

In [ ]:
# Z score normalization, since this feeds into the model
X_pos_lime = X[Y==1] # The X values of all of the "Unhealthy patients"
X_pos_lime = (X_pos_lime - X_t_mean) / X_t_std

In [ ]:
# 1. Define cost function
def K_means_cost(centroids,X_vals,print_freq=False):
  sum = 0
  freq = np.zeros((centroids.shape[0],1))
  for i in range(X_vals.shape[0]):
    dist = np.sum((centroids-X_vals[i,:])**2,axis=1,keepdims=True)**.5
    sum+=min(dist)
    freq+=(dist==min(dist))
  if(print_freq):
    print(freq)
  return (sum/X_vals.shape[0])[0], freq

In [ ]:
# 2. Define cluster candidates
cluster_choices = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,20,25,30,40,50]
costs = []
for i in cluster_choices:
  k_means_testing = KMeans(n_clusters=i, random_state=0)
  k_means_testing.fit(X_pos_clustering)
  centroids_testing = k_means_testing.cluster_centers_
  # 3. Find the cost of each model
  cost, freq = K_means_cost(centroids_testing,X_pos_clustering)
  costs.append(cost)

# 4. Plot it!
plt.scatter(cluster_choices,costs)
plt.plot(cluster_choices,costs)
plt.xlabel("Number of Clusters")
plt.ylabel("Cost")
plt.title("Elbow at K = 10")
plt.savefig("elbow_method.png")
plt.show()

In [ ]:
# Define K-Means Clustering Model
num_clusters = 10
K_Means = KMeans(n_clusters=num_clusters,random_state=0)
K_Means.fit(X_pos_clustering) # Fit the K means clustering algorithm to only the positive data

# Display the X values of the centroids
print("These are the centroids in the original normalized data: \n")
centroids = K_Means.cluster_centers_*X_div # These are the centroids in the original data
centroids = pd.DataFrame(centroids, columns=df.columns[:-1], index=list(range(1,num_clusters+1)))
display(centroids.T)

In [ ]:
cluster_assignments = [[int(K_Means.predict(X_pos_clustering[i].reshape(1,-1))), X_pos_lime[i]] for i in range(len(X_pos_clustering))]

In [ ]:
clusters_dict = {"Cluster " + str(i): np.array([j[1] for j in cluster_assignments if j[0] == i]) for i in range(num_clusters)}

In [ ]:
def nn_predict(data):
  pred = model_nn.predict(data).reshape(-1)
  probs = np.array(list(zip(1-pred, pred)))
  return probs

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_t,
    feature_names=df.columns[:-1],
    class_names=['no', 'yes'],
    mode='classification')

In [ ]:
cols = list(df.columns[:-1])
impact = {}

for ii in range(num_clusters):
  impact["Cluster " + str(ii)] = {col: [] for col in cols}

  for i in range(clusters_dict["Cluster " + str(ii)].shape[0]):
    exp = explainer.explain_instance(
      data_row=clusters_dict["Cluster " + str(ii)][i], 
      predict_fn=nn_predict, num_features=18)
    features = exp.as_map()
    
    for j in list(features.values())[0]:
      impact["Cluster " + str(ii)][cols[j[0]]].append(j[1])

In [ ]:
pickle.dump(impact, open("impact.dat", "wb"))

In [ ]:
impact = pickle.load(open("/content/impact.dat", "rb"))

In [ ]:
def sampling_distribution(data, sample_size):
  np.random.seed(0)
  np.random.shuffle(data)
  means = []
  for i in range(int(len(data) / sample_size - 1)):
    means.append(np.mean(data[sample_size*i:sample_size*(i+1)]))
  means.append(np.mean(data[sample_size*(i+1):]))
  return np.array(means)

In [ ]:
fig = plt.figure(figsize=(20,40))

nbins = 10
colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k', 'tab:brown', 'tab:orange', 'tab:gray']
num_clusters = 10
counter = 1
bimodal = ["Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]
n = 4
x_labels = [
            [0,1,2,3,4,5,6],
            [-60,-40,-20,0,20,-40],
            [0.5,1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5],
            [0,1,2,3,4,5,6,7],
            [19,19.5,20.0,20.5,21.0,21.5,22.0],
            [0,2,4,6,8,10],
            [6.5,7,7.5,8,8.5,9,9.5,10],
            [0,2,4,6,8,10,12,14],
            [6,6.5,7,7.5,8,8.5,9,9.5,10],
            [16,17,18,19,20,21],
            [11.5,12,12.5,13,13.5,14,14.5,15,15.5],
            [6,7,8,9,10,11,12],
            [1,2,3,4,5,6],
            [-0.5,0,0.5,1,1.5,2],
            [2,2.5,3,3.5,4,4.5,5,5.5,6],
            [4,5,6,7,8,9,10],
            [15.5,16,16.5,17,17.5,18,18.5,19,19.5],
            [7.5,8,8.5,9,9.5,10,10.5]
]
for i in list(df.columns[:-1]):
  ax = fig.add_subplot(6, 3, counter, projection='3d')
  max_range_x = 0
  xs_track = np.array(())
  for j in range(num_clusters):
    if i in bimodal:
      subdict = impact["Cluster " + str(j)]
      distribution = sampling_distribution(np.abs(subdict[i]), sample_size=n)
      hist, bins = np.histogram(distribution, bins=10)
    else:  
      subdict = impact["Cluster " + str(j)]
      ys = np.abs(subdict[i])
      # ys = np.random.normal(loc=locs[j], scale=10, size=2000)
      hist, bins = np.histogram(ys, bins=nbins)
    hist = hist/np.sum(hist)
    xs = (bins[:-1] + bins[1:])/2
    xs = xs * 1000 # otherwise plotting won't work 
    range_x = np.abs(np.max(xs) - np.min(xs))
    if range_x > max_range_x:
      max_range_x = range_x
      xs_track = xs
    ax.bar(xs, hist, zs=j, zdir='y', color=colors[j], ec=colors[j], alpha=0.8, label="Cluster " + str(j))
  # print(xs_track.min(), xs_track.max())
  if i == "RestingBP":
    ax.legend(bbox_to_anchor=(1.3,0.8))
  ax.set_xlabel('Probability')
  # ax.set_xticklabels([round(i,3) for i in (xs_track/1000)]) # adjusting tick labels back  
  ax.set_yticks(np.arange(10))
  ax.set_ylabel('Cluster No.')
  ax.set_zlabel('Frequency')
  ax.set_title("Impact of " + str(i) + " Across Clusters", pad=20)
  ax.set_xticklabels([str(k) for k in x_labels[counter-1]])
  # labels = [str(np.array(item.get_text()).astype(float)/10) for item in ax.get_xticklabels()]
  # ax.set_xticklabels(labels)
  
  counter += 1
plt.subplots_adjust(wspace=0, hspace=0.05)
plt.savefig("impact_all_clusters_3D.pdf", bbox_inches='tight')
plt.show()
  

In [ ]:
for i in range(num_clusters):
  print(len(impact["Cluster " + str(i)]["Age"]))

In [ ]:
fig, ax = plt.subplots(6,3,figsize=(15,20))
coords = [(i,j) for i in range(6) for j in range(3)]
counter = 0 
bimodal = ["Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]
n = 4
num_clusters = 10

for i in list(df.columns[:-1]):
  for j in range(num_clusters):
    if i in bimodal:
      subdict = impact["Cluster " + str(j)]
      distribution = sampling_distribution(np.abs(subdict[i]), sample_size=n)
      hist, bins = np.histogram(distribution, bins=10)
      hist = hist/np.sum(hist)
      xs = (bins[:-1] + bins[1:])/2
      xs = xs * 100
      ax[coords[counter]].bar(xs, hist, label="Cluster " + str(j))
    else:
      subdict = impact["Cluster " + str(j)]
      hist, bins = np.histogram(np.abs(subdict[i]), bins=10)
      hist = hist/np.sum(hist)
      xs = (bins[:-1] + bins[1:])/2
      xs = xs * 100
      ax[coords[counter]].bar(xs, hist, label="Cluster " + str(j))
  if i == "RestingBP":
    ax[coords[counter]].legend(bbox_to_anchor=(1.2,1))
  ax[coords[counter]].set_title("Impact of " + str(i) + " Across Clusters")
  ax[coords[counter]].set_xlabel("Probability")
  ax[coords[counter]].set_ylabel("Frequency")
  counter += 1
plt.subplots_adjust(hspace=0.4)
plt.savefig("impact_all_clusters_2D_absolutevalue.pdf", bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(5,2,figsize=(13,20))
coords = [(i,j) for i in range(5) for j in range(2)]
counter = 0
colors = ['b','tab:orange','g','k','tab:purple','tab:pink','tab:brown','tab:gray','y','r',
          'c','m','lime','navy','yellow','peru','mistyrose','skyblue']

for j in range(num_clusters):
  subdict = impact["Cluster " + str(j)]
  for i in list(df.columns[:1]) + list(df.columns[2:-1]):
    hist, bins = np.histogram(np.abs(subdict[i]), bins=10)
    hist = hist/np.sum(hist)
    xs = (bins[:-1] + bins[1:])/2
    xs = xs*100
    ax[coords[counter]].bar(xs, hist, label=i, color=colors[list(df.columns).index(i)], alpha=1)
  if j == 1:
    ax[coords[counter]].legend(bbox_to_anchor=(1.1,1))
  ax[coords[counter]].set_title("Impact of Features on Cluster " + str(j))
  ax[coords[counter]].set_xlabel("Probability")
  ax[coords[counter]].set_ylabel("Frequency")
  counter += 1
plt.subplots_adjust(hspace=0.4)
plt.savefig("impact_all_features_2D_excludingage_absolutevalue.pdf", bbox_inches='tight')
plt.show()

In [ ]:
t_scores = np.zeros((num_clusters, len(df.columns[:-1])))
p_vals = np.zeros((num_clusters, len(df.columns[:-1])))
degs_freedom = np.zeros((num_clusters, len(df.columns[:-1])))
bimodal = ["Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]
n = 4
num_clusters = 10

for i in range(num_clusters):  
  for j in range(len(df.columns[:-1])):
    if str(df.columns[j]) in bimodal:
      subdict = impact["Cluster " + str(i)][str(df.columns[j])]
      distribution = sampling_distribution(np.abs(subdict), sample_size=n)
      mean = np.mean(distribution)
      std = np.std(distribution) * (n ** 0.5)
      t_score = mean / (std / (len(distribution) ** 0.5))
      deg_freedom = len(distribution) - 1
    else: 
      mean = np.mean(impact["Cluster " + str(i)][str(df.columns[j])])
      std = np.std(impact["Cluster " + str(i)][str(df.columns[j])])
      t_score = mean / (std / (len(impact["Cluster " + str(i)][str(df.columns[j])]) ** 0.5))
      deg_freedom = len(list(impact["Cluster " + str(i)].values())[0]) - 1
    p = t.cdf(min(t_score, -t_score), df=deg_freedom)
    t_scores[i][j] = t_score
    p_vals[i][j] = p
    degs_freedom[i][j] = deg_freedom

# t_scores[:,1] and p_vals[:,1] will be NaN because the impact of the "Sex" feature is 0

In [ ]:
pd.DataFrame(p_vals.T)